In [1]:
import math

In [2]:
from constrain.lab.PCR import *

In [3]:
def grouper(iterable, max_diff):
    prev = None
    group = []
    for item in iterable:
        if not prev or item - prev <= max_diff:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group

In [4]:
elong_times = [60,60, 46, 60, 45, 30, 200, 100]
elong_times.sort()
elong_time_max_diff = 10

In [5]:
groups = dict(enumerate(grouper(elong_times,elong_time_max_diff), 1))
groups

{1: [30], 2: [45, 46], 3: [60, 60, 60], 4: [100], 5: [200]}

In [6]:
grouper(elong_times,elong_time_max_diff)

<generator object grouper at 0x7faa330b9040>

In [7]:
from pydna.dseqrecord import Dseqrecord
from pydna.readers import read
from pydna.amplify import pcr
from pydna.primer import Primer
from Bio.SeqRecord import SeqRecord
    
# initialize
middle = 'a'*2000
template = Dseqrecord("tacactcaccgtctatcattatcagcgacgaagcgagcgcgaccgcgagcgcgagcgca"+middle+"caggagcgagacacggcgacgcgagcgagcgagcgatactatcgactgtatcatctgatagcac")
p1 = Primer("tacactcaccgtctatcattatc")
p2 = Primer("cgactgtatcatctgatagcac").reverse_complement()
amplicon = pcr(p1, p2, template)
amplicon.name = 'AMPICON_FOR_TESTING_amplicon_byname_function'
amplicon.annotations['polymerase'] = "OneTaq Hot Start"
det_proc_speed(amplicon)


Amplicon(2123)

In [8]:
amplicon_list = [amplicon]

In [9]:
my_amplicon = amplicon_by_name('AMPICON_FOR_TESTING_amplicon_byname_function', amplicon_list)
len(my_amplicon)

2123

In [10]:
list_of_conc = nanophotometer_concentrations(path = '../tests/files_for_testing/2021-03-29_G8H_CPR_library_part_concentrations.tsv')

assert list_of_conc[0] ==142.8
assert list_of_conc[1] ==134.5

assert list_of_conc[-2] ==17.5
assert list_of_conc[-1] ==39.9

In [11]:

pcr_volumes_df = pcr_volumes(vol_p_reac = 20, 
        no_of_reactions = 3,
        standard_reagents = ["Template", "Primer 1", "Primer 2", "H20", "Pol"],
        standard_volumes = [1, 2.5, 2.5, 19, 25])

pcr_volumes_df

,vol_p_reac,vol_p_3_reac
Template,0.4,1.2
Primer 1,1.0,3.0
Primer 2,1.0,3.0
H20,7.6,22.8
Pol,10.0,30.0
Total,20.0,60.0


In [12]:
round(pcr_volumes_df.iloc[0]['vol_p_3_reac'],2)

1.2

In [13]:
# template
assert pcr_volumes_df.iloc[0]['vol_p_reac'] == 0.4
assert round(pcr_volumes_df.iloc[0]['vol_p_3_reac'],2) == 1.2

# h2o
assert pcr_volumes_df.iloc[3]['vol_p_reac'] == 7.6
assert round(pcr_volumes_df.iloc[3]['vol_p_3_reac'],2) == 22.8

# total
assert pcr_volumes_df.iloc[5]['vol_p_reac'] == 20.0
assert round(pcr_volumes_df.iloc[5]['vol_p_3_reac'],2) == 60.0


In [14]:

middle = 'a'*2000
template = Dseqrecord("tacactcaccgtctatcattatcagcgacgaagcgagcgcgaccgcgagcgcgagcgca"+middle+"caggagcgagacacggcgacgcgagcgagcgagcgatactatcgactgtatcatctgatagcac")
p1 = Primer("tacactcaccgtctatcattatc")
p2 = Primer("cgactgtatcatctgatagcac").reverse_complement()
amplicon = pcr(p1, p2, template)
amplicon.name = 'AMPICON_FOR_TESTING_amplicon_byname_function'
amplicon.annotations['polymerase'] = "OneTaq Hot Start"
det_proc_speed(amplicon)


program = simple_PCR_program(amplicon)

assert program[1:5] == '98°C'
assert program[35:39] == '61.0'
assert program[61:65] == '72°C'

In [15]:
program[61:65]

'72°C'

In [16]:
polymerase = "OneTaq Hot Start"

middle = 'c'*100
template = Dseqrecord("tacactcaccgtctatcattatcagcgacgaagcgagcgcgaccgcgagcgcgagcgca"+middle+"caggagcgagacacggcgacgcgagcgagcgagcgatactatcgactgtatcatctgatagcac")
p1 = Primer("tacactcaccgtctatca")
p2 = Primer("cgactgtatcatctgatagcac").reverse_complement()
amplicon1 = pcr(p1, p2, template)
amplicon1.name = 'AMPLICON1'
amplicon1.annotations['polymerase'] = "OneTaq Hot Start"
det_proc_speed(amplicon1)
det_elon_time(amplicon1)


middle = 'a'*200
template = Dseqrecord("tacactcaccgtctatcattatcagcgacgaagcgagcgcgaccgcgagcgcgagcgca"+middle+"caggagcgagacacggcgacgcgagcgagcgagcgatactatcgactgtatcatctgatagcac")
p1 = Primer("tacactcaccgtctatca")
p2 = Primer("cgactgtatcatctgatagcac").reverse_complement()
amplicon2 = pcr(p1, p2, template)
amplicon2.name = 'AMPLICON2'
amplicon2.annotations['polymerase'] = "OneTaq Hot Start"
det_proc_speed(amplicon2)
det_elon_time(amplicon2)

amplicon1_program = simple_PCR_program(amplicon1)
amplicon2_program = simple_PCR_program(amplicon2)


In [17]:
amplicons = [amplicon1, amplicon2]
amplicons

[Amplicon(223), Amplicon(323)]

In [18]:
thermal_cyclers = det_no_of_thermal_cyclers(amplicons, polymerase='Q5 Hot Start') # pol
thermal_cyclers

,tas,elong_times,amplicons
0,59,20,"AMPLICON1, AMPLICON2"


In [19]:
assert thermal_cyclers.iloc[0]['tas'] == 59
assert thermal_cyclers.iloc[0]['elong_times'] == 20
assert thermal_cyclers.iloc[0]['amplicons'] == 'AMPLICON1, AMPLICON2'
